## 로컬 환경에서 PDF 파일 RAG 검색하기 3단계

__step5__
- PDF 문서 여러개 로드하여 임베딩 후 csv 파일로 저장
- csv 파일을 랭체인 FAISS 인덱싱하여 인덱스 파일 생성 및 저장
- 랭체인 프레임워크 적용하여 llm 검색 까지 구현
- 파일 3개 인덱싱 후 1개 추가하여 llm 검색 하기 구현

- 추가 파일 인덱스 병합하지 않고 각각 저장하는 코드 


In [1]:
import os
import csv
import json
import ast
import faiss
import numpy as np
import pandas as pd
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

# 1. 문서 로드
loader = DirectoryLoader(
    'data',
    glob='*.pdf',
    loader_cls=PyMuPDFLoader
)
docs = loader.load()

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3. 임베딩을 위한 클래스 생성
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]

# 임베딩 모델
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")

# 4. 임베딩 저장 함수
def save_embeddings_to_csv(documents, embedding_model, folder_path):
    os.makedirs(folder_path, exist_ok=True)
    
    file_docs = {}
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf', '')
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)
    
    for file_name, docs in file_docs.items():
        full_path = os.path.join(folder_path, f"{file_name}.csv")
        
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])
        
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["document", "embedding"])
            
            for doc, embedding in zip(docs, embeddings):
                writer.writerow([doc.page_content, json.dumps(embedding)])  # JSON 형식으로 저장
        
        print(f"✅ 임베딩 데이터가 {full_path} 파일에 저장되었습니다.")

# 임베딩 저장 함수 실행
save_embeddings_to_csv(split_documents, embedding_model, 'csv/')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


✅ 임베딩 데이터가 csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
✅ 임베딩 데이터가 csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.


In [2]:
# 5. CSV에서 임베딩 로드 및 FAISS 인덱스 생성
def load_csv_embeddings(folder_path):
    data_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            
            if "document" in df.columns and "embedding" in df.columns:
                documents, embeddings, metadatas = [], [], []
                for _, row in df.iterrows():
                    text = row["document"]
                    try:
                        embedding = json.loads(row["embedding"])  # JSON 로드 방식으로 변경
                        if isinstance(embedding, list):
                            embeddings.append(np.array(embedding, dtype=np.float32))
                            documents.append(text)
                            metadatas.append({"source": filename})
                        else:
                            print(f"⚠️ {filename}의 임베딩 형식이 올바르지 않습니다!")
                    except Exception as e:
                        print(f"⚠️ {filename}에서 임베딩 변환 오류: {e}")
                data_dict[filename.replace('.csv', '')] = (documents, embeddings, metadatas)
    
    return data_dict

# FAISS 인덱스 생성 및 저장
def create_faiss_indexes(data_dict, save_path):
    os.makedirs(save_path, exist_ok=True)
    
    for file_name, (documents, embeddings, metadatas) in data_dict.items():
        vector_store = FAISS.from_texts(texts=documents, embedding=embedding_model, metadatas=metadatas)
        faiss_index_path = os.path.join(save_path, file_name)
        vector_store.save_local(faiss_index_path)
        print(f"✅ {file_name}에 대한 FAISS 인덱스 저장 완료: {faiss_index_path}")

In [3]:
# 실행
if __name__ == "__main__":
    data_folder = "./csv"
    faiss_index_folder = "./faiss_index"
    
    data_dict = load_csv_embeddings(data_folder)
    print(f"📄 {len(data_dict)}개의 파일 로드 완료!")
    
    if data_dict:
        create_faiss_indexes(data_dict, faiss_index_folder)
    else:
        print("⚠️ 문서 또는 임베딩 데이터가 없습니다. FAISS 인덱스를 생성할 수 없습니다!")

📄 2개의 파일 로드 완료!
✅ SPRI_AI_Brief_2023년12월호_F에 대한 FAISS 인덱스 저장 완료: ./faiss_index/SPRI_AI_Brief_2023년12월호_F
✅ AI기반_인파분석플랫폼구축_제안서에 대한 FAISS 인덱스 저장 완료: ./faiss_index/AI기반_인파분석플랫폼구축_제안서


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

from langchain_openai import ChatOpenAI
import os

# ✅ Step 6: 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean, and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

# Step 7: 언어 모델 (LLM) 생성
llm = Ollama(model="llama3.2")  


# ✅ Step 4: 모든 FAISS 인덱스를 로드하는 함수
def load_all_faiss_indexes(index_folder):
    faiss_indexes = {}

    for file_name in os.listdir(index_folder):
        index_path = os.path.join(index_folder, file_name)
        if os.path.isdir(index_path):  # 폴더 형태의 FAISS 인덱스인지 확인
            try:
                faiss_indexes[file_name] = FAISS.load_local(
                    index_path,
                    embedding_model,
                    allow_dangerous_deserialization=True  # 보안 옵션 추가
                )
                print(f"✅ {file_name} 인덱스 로드 완료!")
            except Exception as e:
                print(f"⚠️ {file_name} 인덱스 로드 실패: {e}")
    
    return faiss_indexes

# ✅ Step 5: 모든 FAISS 인덱스를 검색하는 함수
def search_across_all_indexes(question, faiss_indexes, top_k=5):
    all_docs = []

    # 모든 FAISS 인덱스에서 검색
    for index_name, index in faiss_indexes.items():
        retriever = index.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
        docs = retriever.get_relevant_documents(question)  

        # 문서 출처 정보 추가
        for doc in docs:
            doc.metadata["source"] = index_name
        all_docs.extend(docs)

    if not all_docs:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # ✅ Step 6: 검색된 문서들을 기반으로 컨텍스트 생성
    context = "\n\n".join([doc.page_content for doc in all_docs])
    
    # ✅ Step 7: LLM을 사용해 답변 생성
    formatted_prompt = prompt.format(context=context, question=question)
    answer = llm.invoke(formatted_prompt)

    # ✅ Step 8: 검색된 문서들의 출처 정리
    sources = "\n".join(set([doc.metadata.get("source", "알 수 없음") for doc in all_docs]))

    return f"✅ 답변: {answer.strip()}\n📁 출처:\n{sources}"

# ✅ 실행 코드
if __name__ == "__main__":
    faiss_index_folder = "./faiss_index"  # FAISS 인덱스 저장 폴더
    question = "삼성전자가 만든 AI의 이름은?"

    # ✅ Step 1: 모든 FAISS 인덱스 로드
    faiss_indexes = load_all_faiss_indexes(faiss_index_folder)

    # ✅ Step 2: 질문 수행
    response = search_across_all_indexes(question, faiss_indexes)
    
    # ✅ Step 3: 결과 출력
    print(response)

✅ SPRI_AI_Brief_2023년12월호_F 인덱스 로드 완료!
✅ AI기반_인파분석플랫폼구축_제안서 인덱스 로드 완료!
✅ 답변: 챌레오입니다.
📁 출처:
SPRI_AI_Brief_2023년12월호_F
AI기반_인파분석플랫폼구축_제안서


In [6]:
def search_faiss_index(query, index_folder, file_name, top_k=5):
    index_path = os.path.join(index_folder, file_name)
    
    # FAISS 인덱스 로드 시 allow_dangerous_deserialization 옵션 추가
    vector_store = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
    
    print(f"🔍 '{query}' 검색 결과:")
    for i, result in enumerate(results):
        print(f"{i+1}. {result.page_content} (출처: {result.metadata['source']})")
    
    return results

# search_faiss_index("포항에서 열리는 축제의 이름은 무엇인가?", "./faiss_index", "AI기반_인파분석플랫폼구축_제안서")
search_faiss_index("삼성에서 만든 AI의 이름은 무엇인가?", "./faiss_index", "SPRI_AI_Brief_2023년12월호_F")
# search_faiss_index("경기도 인구정책사업에서 현행 유지 평가를 받은 사업의 개수는?", "./faiss_index", "2024년 경기도 인구영향평가_편집본")

🔍 '삼성에서 만든 AI의 이름은 무엇인가?' 검색 결과:
1. SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 

[Document(id='35bbb80f-2959-4bfd-b489-0afd3a96506f', metadata={'source': 'SPRI_AI_Brief_2023년12월호_F.csv'}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성